In [1]:
%run ./check_DOVS_METHODS.ipynb

In [2]:
from importlib import reload
import sys, os
import re
from pathlib import Path
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version

import copy
import itertools
import adjustText

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
from DOVSAudit import DOVSAudit
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import PDFMerger
import Plot_General
import Plot_Box_sns
import Plot_Hist
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSingleSlicer as DFSingleSlicer
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet

# Analyze collected data

## AMI NonVee

In [ ]:
import warnings
warnings.filterwarnings("ignore", "use_inf_as_na")

In [ ]:
date_0 = '2025-01-12'
date_1 = '2025-01-18'
opco   = 'pso'
#-------------------------
# Whether or not to skip pre-existing results
skip_prex_results = True
perform_plotting  = True
build_summary_dfs = True
#-------------------------
save_dir_base = r'C:\Users\s346557\Documents\LocalData\dovs_check'
save_subdir = f"{date_0.replace('-','')}_{date_1.replace('-','')}"
#-----
base_dir     = os.path.join(save_dir_base, save_subdir, opco)
base_dir_ami = os.path.join(base_dir, r'AMINonVee')
base_dir_ede = os.path.join(base_dir, r'EndEvents')
#-----
save_dir           = os.path.join(base_dir, r'Results')
dovs_audits_subdir = os.path.join(save_dir, 'dovs_audits')
#-----
assert(os.path.exists(base_dir_ami))
assert(os.path.exists(base_dir_ede))
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
if not os.path.exists(dovs_audits_subdir):
    os.makedirs(dovs_audits_subdir)

In [ ]:
#--------------------------------------------------
calculate_by_PN = True
#-------------------------
# combine_by_PN_likeness_thresh = pd.Timedelta('1 minutes')
combine_by_PN_likeness_thresh = pd.Timedelta('15 minutes')
#-------------------------
expand_outg_search_time_tight = pd.Timedelta('1 hours')
expand_outg_search_time_loose = pd.Timedelta('12 hours')
#-------------------------
use_est_outg_times=False
# use_est_outg_times=True
#-------------------------
use_full_ede_outgs=False
#-------------------------
run_outg_inclusion_assessment=True
#-------------------------
max_pct_PNs_missing_allowed=0
# max_pct_PNs_missing_allowed=20

#--------------------------------------------------
include_suboutg_endpt_plots=True
#-------------------------
n_PNs_w_power_threshold = 95

In [ ]:
#--------------------------------------------------
outg_rec_nb_to_files_dict = DOVSAudit.get_outg_rec_nb_to_files_dict_ami(
    base_dir_dict   = base_dir,
    base_dir_data   = base_dir_ami, 
    rebuild         = False, 
    save_dict       = True
)
all_outg_rec_nbs = list(outg_rec_nb_to_files_dict.keys())
#-------------------------
outg_rec_nb_to_files_ede_dict = DOVSAudit.get_outg_rec_nb_to_files_dict_ede(
    base_dir_dict   = base_dir,
    base_dir_data   = base_dir_ede, 
    rebuild         = False, 
    save_dict       = True
)
all_outg_rec_nbs_ede = list(outg_rec_nb_to_files_ede_dict.keys())

In [ ]:
if perform_plotting:
    # In some cases, keeping all of the PdfPages objects open becomes taxing in terms of memory
    # Therefore, I will save all of the PDFs as separate documents, closing each at the end of each iteration
    #   and collecting all in their respective single files at the end.
    # The following paths are for the final, single files
    #--------------------------------------------------
    res_tmp_subdir = 'TMP_Results'
    res_pdf_path   = os.path.join(save_dir, r'Results.pdf')
    #-------------------------
    res_dovs_beg_tmp_subdir = 'TMP_Results_dovs_beg'
    res_dovs_beg_pdf_path   = Utilities.append_to_path(
        res_pdf_path, 
        '_dovs_beg', 
        ext_to_find='.pdf', 
        append_to_end_if_ext_no_found=False
    )
    #-------------------------
    res_w_endpts_tmp_subdir = 'TMP_Results_w_suboutg_endpt'
    res_w_endpts_pdf_path   = Utilities.append_to_path(
        res_pdf_path, 
        '_w_suboutg_endpt_plots', 
        ext_to_find='.pdf', 
        append_to_end_if_ext_no_found=False
    )
    #--------------------------------------------------
    tmp_subdirs = [
        res_tmp_subdir, 
        res_dovs_beg_tmp_subdir, 
        res_w_endpts_tmp_subdir
    ]
    #-----
    tmp_subdir_paths = Utilities.make_tmp_save_dir(
        base_dir_path = save_dir,
        tmp_dir_name  = tmp_subdirs, 
        return_path   = True
    )

In [ ]:
# Find any pre-existing results
prex_audit_paths = Utilities.find_all_paths(
    base_dir      = os.path.join(save_dir, dovs_audits_subdir), 
    glob_pattern  = r'*.pkl', 
    regex_pattern = None
)
prex_audit_paths = natsorted(prex_audit_paths)
prex_audits      = [Path(x).stem for x in prex_audit_paths]

In [ ]:
if skip_prex_results:
    outg_rec_nbs = list(set(all_outg_rec_nbs).difference(set(prex_audits)))
else:
    outg_rec_nbs = all_outg_rec_nbs

In [ ]:
print(f'len(all_outg_rec_nbs): {len(all_outg_rec_nbs)}')
print(f'len(prex_audits):      {len(prex_audits)}')
print(f'len(outg_rec_nbs):     {len(outg_rec_nbs)}')

In [ ]:
#--------------------------------------------------
outgs_pass = []
outgs_fail = []
#-------------------------
fig_num                           = 0
all_detailed_summary_dfs          = []
all_detailed_summary_dfs_dovs_beg = []
ci_cmi_summary_dfs                = []
warnings_text                     = ''

#--------------------------------------------------
# Build dovs_df
dovs = DOVSOutages(
    df_construct_type         = DFConstructType.kRunSqlQuery, 
    contstruct_df_args        = None, 
    init_df_in_constructor    = True,
    build_sql_function        = DOVSOutages_SQL.build_sql_std_outage, 
    build_sql_function_kwargs = dict(
        outg_rec_nbs    = outg_rec_nbs, 
        field_to_split  = 'outg_rec_nbs', 
        include_premise = True, 
        opco            = opco
    ), 
    build_consolidated        = True
)
dovs_df = dovs.df.copy()

#----------------------------------------------------------------------------------------------------
# Now, iterate through all outages
for i_outg, outg_rec_nb in enumerate(outg_rec_nbs):
    print(f'\n\ti_outg: {i_outg}/{len(outg_rec_nbs)-1}')
    print(f'\toutg_rec_nb = {outg_rec_nb}')
    #--------------------------------------------------
    try:
        audit_i = DOVSAudit(
            outg_rec_nb                   = outg_rec_nb, 
            calculate_by_PN               = calculate_by_PN, 
            combine_by_PN_likeness_thresh = combine_by_PN_likeness_thresh, 
            expand_outg_search_time_tight = expand_outg_search_time_tight, 
            expand_outg_search_time_loose = expand_outg_search_time_loose, 
            use_est_outg_times            = use_est_outg_times, 
            use_full_ede_outgs            = use_full_ede_outgs, 
            run_outg_inclusion_assessment = run_outg_inclusion_assessment, 
            max_pct_PNs_missing_allowed   = max_pct_PNs_missing_allowed, 
            opco                          = opco
        )
        
        audit_i.load_ami_from_csvs(
            paths                          = outg_rec_nb_to_files_dict[outg_rec_nb], 
            slicers                        = None, 
            ami_df_info_dict               = None, 
            run_std_init                   = True, 
            cols_and_types_to_convert_dict = None, 
            to_numeric_errors              = 'coerce', 
            drop_na_rows_when_exception    = True, 
            drop_unnamed0_col              = True, 
            pd_read_csv_kwargs             = None, 
            make_all_columns_lowercase     = False, 
            assert_all_cols_equal          = True, 
            min_fsize_MB                   = None
        )
        #--------------------------------------------------
        if audit_i.ami_df_i.shape[0]==0:
            outgs_fail.append((audit_i.outg_rec_nb, "ami_df_i.shape[0]==0"))
            continue
    
        #-------------------------
        # Need to load dovs before running self assessment below
        audit_i.load_dovs(
            dovs_df           = dovs_df, 
            dovs_df_info_dict = None
        )
        
        if run_outg_inclusion_assessment:
            to_include_i = audit_i.self_assess_outage_inclusion_requirements(max_pct_PNs_missing_allowed, None)
            if not to_include_i:
                print(f'outg_rec_nb={audit_i.outg_rec_nb} did not pass inclusion requirements, skipping!!!!!')
                outgs_fail.append((audit_i.outg_rec_nb, "Inclusion Requirements"))
                continue
        #-------------------------    
        #----------------------------------------------------------------------------------------------------
        if audit_i.outg_rec_nb in outg_rec_nb_to_files_ede_dict.keys():
            audit_i.load_ede_from_csvs(
                paths                          = outg_rec_nb_to_files_ede_dict[audit_i.outg_rec_nb], 
                ede_df_info_dict               = None, 
                run_std_init                   = True, 
                cols_and_types_to_convert_dict = None, 
                to_numeric_errors              = 'coerce', 
                drop_na_rows_when_exception    = True, 
                drop_unnamed0_col              = True, 
                pd_read_csv_kwargs             = None, 
                make_all_columns_lowercase     = False, 
                assert_all_cols_equal          = True, 
                min_fsize_MB                   = None
            )
        #----------------------------------------------------------------------------------------------------
        audit_i.build_best_ests_df()
        #--------------------------------------------------
        # cnsrvtv_out_t_beg/_end are used for placing bounds on the plots generated
        # NOTE: If one did not want to show any data which was thrown out due to overlapping, one would want to
        #         update cnsrvtv_out_t_beg/_end after the identify_dovs_overlaps_from_best_ests procedure below 
        #         (and subsequent trimming of audit_i.best_ests_df)
        cnsrvtv_out_t_beg_end = audit_i.get_cnsrvtv_out_t_beg_end()

        #--------------------------------------------------
        audit_i.identify_overlaps_and_resolve(
            overlaps_dovs_sql_fcn           = DOVSOutages_SQL.build_sql_outage, 
            dovs_df                         = None, 
            t_min_col                       = 'winner_min', 
            t_max_col                       = 'winner_max', 
            keep_col                        = 'keep', 
            overlap_DOVS_col                = 'overlap_DOVS', 
            outg_rec_nb_col_dovs            = 'OUTG_REC_NB', 
            dt_off_ts_full_col_dovs         = 'DT_OFF_TS_FULL', 
            dt_on_ts_col_dovs               = 'DT_ON_TS', 
            overlaps_addtnl_dovs_sql_kwargs = dict(
                CI_NB_min  = 0, 
                CMI_NB_min = 0
            ), 
            overlap_disagree_cols           = ['ovrlp_disagree_typeA', 'ovrlp_disagree_typeB'], 
            unq_idfr_cols                   = ['PN', 'i_outg'], 
            open_beg_col                    = 'open_beg', 
            open_end_col                    = 'open_end'
        )
        #--------------------------------------------------
        audit_i.finalize_analysis()
        #--------------------------------------------------
        audit_i.save(os.path.join(dovs_audits_subdir, f'{audit_i.outg_rec_nb}.pkl'))
        #--------------------------------------------------
        if not audit_i.can_analyze:
            outgs_fail.append((audit_i.outg_rec_nb, "not can_analyze (likely overlapping DOVS)"))
            continue
        #----------------------------------------------------------------------------------------------------
        if build_summary_dfs:
            detailed_summary_df_i = audit_i.get_detailed_summary_df(
                dovs_beg        = False, 
                delta_t_off_cut = pd.Timedelta('5min'), 
                delta_t_on_cut  = pd.Timedelta('5min'), 
                delta_ci_cut    = 3, 
                delta_cmi_cut   = None, 
                n_PNs_w_power_threshold = n_PNs_w_power_threshold, 
            )
            if detailed_summary_df_i.shape[0]>0:
                all_detailed_summary_dfs.append(detailed_summary_df_i)
            #-----
            detailed_summary_df_dovs_beg_i = audit_i.get_detailed_summary_df(
                dovs_beg        = True, 
                delta_t_off_cut = pd.Timedelta('5min'), 
                delta_t_on_cut  = pd.Timedelta('5min'), 
                delta_ci_cut    = 3, 
                delta_cmi_cut   = None, 
                n_PNs_w_power_threshold = n_PNs_w_power_threshold, 
            )
            if detailed_summary_df_dovs_beg_i.shape[0]>0:
                all_detailed_summary_dfs_dovs_beg.append(detailed_summary_df_dovs_beg_i)
            #-----
            warnings_text += audit_i.generate_warnings_text()
            #-------------------------
            ci_cmi_summary_dfs.append(
                pd.DataFrame(
                    dict(
                        outg_rec_nb      = audit_i.outg_rec_nb, 
                        ci_dovs          = audit_i.ci_dovs,   
                        ci_ami           = audit_i.ci, 
                        ci_ami_dovs_beg  = audit_i.ci_dovs_beg, 
                        cmi_dovs         = audit_i.cmi_dovs, 
                        cmi_ami          = audit_i.cmi, 
                        cmi_ami_dovs_beg = audit_i.cmi_dovs_beg
                    ), 
                    index=[len(ci_cmi_summary_dfs)]
                )
            )
        #----------------------------------------------------------------------------------------------------
        # ######################### PLOTTING #########################
        #----------------------------------------------------------------------------------------------------
        if perform_plotting:
            fig, axs = audit_i.plot_results(
                include_dovs_beg_text      = True, 
                name                       = 'AMI', 
                expand_time                = pd.Timedelta('1 hour'), 
                n_PNs_w_power_threshold    = n_PNs_w_power_threshold, 
                fig_num                    = fig_num
            )    
            Plot_General.save_fig(
                fig         = fig, 
                save_dir    = os.path.join(save_dir, res_tmp_subdir), 
                save_name   = f"{audit_i.outg_rec_nb}.pdf", 
                bbox_inches = 'tight'
            )
            if include_suboutg_endpt_plots:
                Plot_General.save_fig(
                    fig         = fig, 
                    save_dir    = os.path.join(save_dir, res_w_endpts_tmp_subdir), 
                    save_name   = f"{audit_i.outg_rec_nb}_0.pdf", 
                    bbox_inches = 'tight'
                )
            fig.clear()
            plt.close(fig)
            fig_num += 1
            
            #----------------------------------------------------------------------------------------------------
            if audit_i.best_ests_means_df_dovs_beg is not None and audit_i.best_ests_means_df_dovs_beg.shape[0]>0:
                fig, axs = audit_i.plot_results_dovs_beg(
                    include_full_alg_text      = True, 
                    name                       = 'AMI w/ DOVS t_beg', 
                    expand_time                = pd.Timedelta('1 hour'), 
                    n_PNs_w_power_threshold    = n_PNs_w_power_threshold, 
                    fig_num                    = fig_num
                )    
                Plot_General.save_fig(
                    fig         = fig, 
                    save_dir    = os.path.join(save_dir, res_dovs_beg_tmp_subdir), 
                    save_name   = f"{audit_i.outg_rec_nb}.pdf", 
                    bbox_inches = 'tight'
                )
                fig.clear()
                plt.close(fig)
                fig_num += 1
            
            #----------------------------------------------------------------------------------------------------
            if include_suboutg_endpt_plots:
                fig_axs = audit_i.plot_zoomed_endpts(
                    fig_num     = fig_num
                )
                if fig_axs is not None:
                    fig = fig_axs[0]
                    axs = fig_axs[1]
                    #-------------------------
                    Plot_General.save_fig(
                        fig         = fig, 
                        save_dir    = os.path.join(save_dir, res_w_endpts_tmp_subdir), 
                        save_name   = f"{audit_i.outg_rec_nb}_1.pdf", 
                        bbox_inches = 'tight'
                    ) 
                    fig.clear()
                    plt.close(fig)
                    fig_num += 1
    
        #----------------------------------------------------------------------------------------------------
        outgs_pass.append(audit_i.outg_rec_nb)

    except:
        outgs_fail.append((audit_i.outg_rec_nb, "Unknown"))
        
#----------------------------------------------------------------------------------------------------
if build_summary_dfs:
    detailed_summary_df          = Utilities_df.concat_dfs(
        dfs                  = all_detailed_summary_dfs, 
        axis                 = 0, 
        make_col_types_equal = False
    )
    detailed_summary_df_dovs_beg = Utilities_df.concat_dfs(
        dfs                  = all_detailed_summary_dfs_dovs_beg, 
        axis                 = 0, 
        make_col_types_equal = False
    )
    #-------------------------
    detailed_summary_df = DOVSAudit.sort_detailed_summary_df(
        detailed_summary_df = detailed_summary_df, 
        how                 = 'abs_delta_ci_cmi', 
    )
    #-----
    detailed_summary_df_dovs_beg = DOVSAudit.sort_detailed_summary_df(
        detailed_summary_df = detailed_summary_df_dovs_beg, 
        how                 = 'abs_delta_ci_cmi', 
    )
    #-------------------------
    ci_cmi_summary_df            = Utilities_df.concat_dfs(
        dfs                  = ci_cmi_summary_dfs, 
        axis                 = 0, 
        make_col_types_equal = False
    )

In [ ]:
print(f"#OUTG_REC_NBs = {len(outg_rec_nbs)}")
print(f"\tpass: {len(outgs_pass)}")
print(f"\tfail: {len(outgs_fail)}")

In [ ]:
if perform_plotting:
    PDFMerger.merge_all_pdfs_in_dir(
        dir_to_merge = os.path.join(save_dir, res_tmp_subdir), 
        output_path  = res_pdf_path, 
    )
    #-----
    PDFMerger.merge_all_pdfs_in_dir(
        dir_to_merge = os.path.join(save_dir, res_dovs_beg_tmp_subdir), 
        output_path  = res_dovs_beg_pdf_path, 
    )
    #-----
    PDFMerger.merge_all_pdfs_in_dir(
        dir_to_merge = os.path.join(save_dir, res_w_endpts_tmp_subdir), 
        output_path  = res_w_endpts_pdf_path, 
    )
    #-------------------------
    # Utilities.del_tmp_save_dir(
    #     base_dir_path = save_dir,
    #     tmp_dir_name  = tmp_subdirs
    # )

In [ ]:
if build_summary_dfs:
    ci_cmi_summary_df['ci_dovs']         = ci_cmi_summary_df['ci_dovs'].astype(float)
    ci_cmi_summary_df['ci_ami']          = ci_cmi_summary_df['ci_ami'].astype(float)
    ci_cmi_summary_df['ci_ami_dovs_beg'] = ci_cmi_summary_df['ci_ami_dovs_beg'].astype(float)
    #-----
    ci_cmi_summary_df['delta_ci_dovs_ami']  = ci_cmi_summary_df['ci_dovs']-ci_cmi_summary_df['ci_ami']
    ci_cmi_summary_df['delta_cmi_dovs_ami'] = ci_cmi_summary_df['cmi_dovs']-ci_cmi_summary_df['cmi_ami']
    #-----
    ci_cmi_summary_df['delta_ci_dovs_ami_dovs_beg']  = ci_cmi_summary_df['ci_dovs']-ci_cmi_summary_df['ci_ami_dovs_beg']
    ci_cmi_summary_df['delta_cmi_dovs_ami_dovs_beg'] = ci_cmi_summary_df['cmi_dovs']-ci_cmi_summary_df['cmi_ami_dovs_beg']
    #-----
    # For plotting purposes, make a outg_rec_in column which is simply 0 to delta_df.shape[0]-1
    ci_cmi_summary_df['outg_rec_int'] = range(ci_cmi_summary_df.shape[0])
    #-----

In [ ]:
if build_summary_dfs:
    detailed_summary_df.to_pickle(os.path.join(save_dir, r'detailed_summary.pkl'))
    detailed_summary_df_dovs_beg.to_pickle(os.path.join(save_dir, r'detailed_summary_dovs_beg.pkl'))
    ci_cmi_summary_df.to_pickle(os.path.join(save_dir, r'ci_cmi_summary.pkl'))
    #-----
    detailed_summary_df.to_csv(os.path.join(save_dir, r'detailed_summary.csv'))
    detailed_summary_df_dovs_beg.to_csv(os.path.join(save_dir, r'detailed_summary_dovs_beg.csv'))
    ci_cmi_summary_df.to_csv(os.path.join(save_dir, r'ci_cmi_summary.csv'))
    #-----
    # For Mico and Amanda
    detailed_summary_df_dovs_beg.to_csv(os.path.join(save_dir, f'detailed_summary_dovs_beg_{save_subdir}.csv'))
    #-----
    with open(os.path.join(save_dir, r'warnings.txt'), 'w') as f:
        f.write(warnings_text)

In [ ]:
assert(0)

# ===========================================================

In [12]:
df_ap  = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_check\20250112_20250118\ap\Results\detailed_summary_dovs_beg.pkl')
df_im  = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_check\20250112_20250118\im\Results\detailed_summary_dovs_beg.pkl')
df_oh  = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_check\20250112_20250118\oh\Results\detailed_summary_dovs_beg.pkl')
df_pso = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_check\20250112_20250118\pso\Results\detailed_summary_dovs_beg.pkl')
df_swp = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_check\20250112_20250118\swp\Results\detailed_summary_dovs_beg.pkl')

In [13]:
with pd.ExcelWriter(r'C:\Users\s346557\Documents\LocalData\dovs_check\20250112_20250118\AllOPCOs\Results\detailed_summary_dovs_beg_20250112_20250118.xlsx') as writer:  
    df_ap.reset_index().to_excel(writer, sheet_name='ap')
    df_im.reset_index().to_excel(writer, sheet_name='im')
    df_oh.reset_index().to_excel(writer, sheet_name='oh')
    df_pso.reset_index().to_excel(writer, sheet_name='pso')
    df_swp.reset_index().to_excel(writer, sheet_name='swp')

In [14]:
PDFMerger.merge_list(
    files_to_merge = [
        r'C:\Users\s346557\Documents\LocalData\dovs_check\20250112_20250118\ap\Results\Results_dovs_beg.pdf', 
        r'C:\Users\s346557\Documents\LocalData\dovs_check\20250112_20250118\im\Results\Results_dovs_beg.pdf', 
        r'C:\Users\s346557\Documents\LocalData\dovs_check\20250112_20250118\oh\Results\Results_dovs_beg.pdf', 
        r'C:\Users\s346557\Documents\LocalData\dovs_check\20250112_20250118\pso\Results\Results_dovs_beg.pdf', 
        r'C:\Users\s346557\Documents\LocalData\dovs_check\20250112_20250118\swp\Results\Results_dovs_beg.pdf', 
    ], 
    output_path    = r'C:\Users\s346557\Documents\LocalData\dovs_check\20250112_20250118\AllOPCOs\Results\Results_dovs_beg_20250112_20250118.pdf'
)